In [1]:
from ray import tune
from ray import air
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from hyperopt import hp
from ray.tune.search.hyperopt import HyperOptSearch

In [2]:
import time
def train(config):
    for epoch in range(10):
        loss = 1/(epoch+1) + (0.001-config["lr"])**2 + config["weight_decay"]
        tune.report(loss=loss)

def train_noray(config):
    for epoch in range(10):
        loss = 1/(epoch+1) + (0.001-config["lr"])**2 + config["weight_decay"]
        print(loss)

In [3]:
train_noray({"lr":0.0048,"weight_decay":0})

1.00001444
0.50001444
0.3333477733333333
0.25001444
0.20001444000000002
0.16668110666666666
0.14287158285714285
0.12501444
0.11112555111111111
0.10001444000000001


In [4]:
config = {
    "lr": tune.loguniform(1e-4, 1e-1),
    "weight_decay": tune.uniform(0,0.3)
}

scheduler = ASHAScheduler(
    max_t=10,
    grace_period=1,
    reduction_factor=2)

hyperopt_search = HyperOptSearch(metric="loss", mode="min")

tuner = tune.Tuner(
    tune.with_resources(
        tune.with_parameters(train),
        resources={"cpu": 2, "gpu": 1}
    ),
    tune_config=tune.TuneConfig(
        search_alg=hyperopt_search,
        metric="loss",
        mode="min",
        scheduler=scheduler,
        num_samples=100,
    ),
    param_space=config,
)
results = tuner.fit()

best_result = results.get_best_result("loss", "min")

2023-04-21 23:29:24,687	INFO worker.py:1553 -- Started a local Ray instance.
/local/scratch-3/fwe21/miniconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Trial name,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,loss,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_1fefd82b,2023-04-21_23-29-29,False,,1e82ddcfe54c4917acf63ea395d472a7,beyla.cl.cam.ac.uk,1,1.08105,128.232.65.165,1805216,0.000281334,0.000281334,0.000281334,1682116169,0,,1,1fefd82b,0.0054009


2023-04-21 23:29:32,539	INFO tune.py:798 -- Total run time: 6.16 seconds (4.75 seconds for the tuning loop).


In [5]:
best_result

Result(metrics={'loss': 0.10242259753980555, 'done': True, 'trial_id': '37af125c', 'experiment_tag': '13_lr=0.0001,weight_decay=0.0024'}, error=None, log_dir=PosixPath('/home/fwe21/ray_results/train_2023-04-21_23-29-22/train_37af125c_13_lr=0.0001,weight_decay=0.0024_2023-04-21_23-29-30'))